In [ ]:
! pip install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

def platypus_training():
    data = load_dataset("garage-bAInd/Open-Platypus", split="train")
    data_df = data.to_pandas()
    data_df["text"] = data_df[["instruction", "output"]].apply(lambda x: x["instruction"] + " " + x["output"], axis=1)
    data_df.drop(["instruction", "output"], axis=1, inplace=True)
    data = Dataset.from_pandas(data_df)
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-2-7b-hf", quantization_config=bnb_config, device_map={"": 0}
    )

    model.config.use_cache=False
    model.config.pretraining_tp=1
    peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules = ["gate_proj" , "down_proj", "up_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    training_arguments = TrainingArguments(
        output_dir="platypus_llama_7b",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="steps",
        save_steps = 50,
        save_total_limit = 100,
        logging_steps=10,
        num_train_epochs=1,
        max_steps=110,
        fp16=True,
        push_to_hub=True
    )

    trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
    )
    trainer.train()
    trainer.push_to_hub()
    trainer.save_model("platypus_llama_7b")

    output_dir = os.path.join("platypus_llama_7b", "final_checkpoint")
    trainer.model.save_pretrained(output_dir)

if __name__ == "__main__":
    platypus_training()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/24926 [00:00<?, ? examples/s]

Cloning https://huggingface.co/Vasanth/platypus_llama_7b into local empty directory.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.225400
20,1.166900
30,1.088600
40,1.067700
50,1.037500
60,1.062700
70,0.966000
80,1.084800
90,1.018700


Step,Training Loss
10,1.225400
20,1.166900
30,1.088600
40,1.067700
50,1.037500
60,1.062700
70,0.966000
80,1.084800
90,1.018700
100,1.005500


Upload file adapter_model.bin:   0%|          | 1.00/88.6M [00:00<?, ?B/s]

Upload file runs/Aug17_23-44-43_afd7057d74a0/events.out.tfevents.1692315953.afd7057d74a0.1023.0:   0%|        …

To https://huggingface.co/Vasanth/platypus_llama_7b
   e5c6c6b..c6f0a41  main -> main

   e5c6c6b..c6f0a41  main -> main

To https://huggingface.co/Vasanth/platypus_llama_7b
   c6f0a41..65bdb25  main -> main

   c6f0a41..65bdb25  main -> main

To https://huggingface.co/Vasanth/platypus_llama_7b
   65bdb25..71f083d  main -> main

   65bdb25..71f083d  main -> main

To https://huggingface.co/Vasanth/platypus_llama_7b
   71f083d..716d0f0  main -> main

   71f083d..716d0f0  main -> main



In [ ]:
! cp -r /content/platypus_llama_7b /content/drive/MyDrive/

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# from peft import AutoPeftModelForCausalLM
# from transformers import BitsAndBytesConfig
# import os
# import torch

# model = AutoPeftModelForCausalLM.from_pretrained("/content/drive/MyDrive/platypus_llama_7b/final_checkpoint", device_map={"": 0})
# model = model.merge_and_unload()
# output_merged_dir = os.path.join("/content/drive/MyDrive/platypus_llama_7b", "final_merged_checkpoint")
# model.save_pretrained(output_merged_dir, safe_serialization=True)